# Sidekick - Formal Definition of the Problems

## Notations
 - Let us denote $\mathbf{v}_{i:j}$, $i \leq j$ the subvector $\mathbf{u} = [v_i, ..., v_j]^T$ consisting of the elements $v_i$ to $v_j$ of the vector $\mathbf{v}$.

 - Let us denote $(v_{i:j}, i \leq j)$ the subsequence $(u_n, n = i, ..., j) = (v_i, ..., v_j)$ consisting of the elements $v_i$ to $v_j$ of the sequence $(v_n, n = 1, ..., N)$.

## Problems

### Single-Project Regression
#### Model
We are approaching the problem as time series regression, considering only one project. Our dataset $\mathcal{D} = \left\{ (x_i, y_i) \mid i = 1, ..., T \right\}$ consists of $T$ observations, with $x_i$ the time index of the amount of money $y_i$. Hence, we have $X = [1, ..., T]^T$ a $(T \times 1)$ matrix of time indices and $\mathbf{y} = [y_1, ..., y_T]^T$ a vector of observed values. We model the pledged money $f(\mathbf{x})$ at time indices $\mathbf{x}$ as a Gaussian Process:

$$f(\mathbf{x}) \sim GP \left( m(\mathbf{x}), k(\mathbf{x}, \mathbf{x}') \right). $$

Our goal is to predict the future values of the pledged money $\mathbf{f}_* = \mathbf{f}_{t:T} = f(X_{t:T})$ at future time indices $X_* = X_{t:T} = [t, ..., T]^T$ after observing the values $\mathbf{y} = \mathbf{y}_{1:t} = [y_1, ..., y_t]^T$ at time indices $X = X_{1:t} = [1, ..., t]^T$. In the GP framework, we can compute this prediction using

$$\mathbf{f}_* \mid X, \mathbf{y}, X_* \sim \mathcal{N}\left(\overline{\mathbf{f}}_*, \text{ cov}(\mathbf{f}_*)  \right) \\
\overline{\mathbf{f}}_* = K(X_*, X) \left[ K(X, X) + \sigma_n^2I \right]^{-1}\mathbf{y} \\
\text{cov}(\mathbf{f}_*) = K(X_*, X_*) - K(X_*, X)\left[ K(X, X) + \sigma_n^2I \right]^{-1}K(X, X_*).
$$ 

Finally, the kernel's (hyper)parameters $\theta_*$ are learned by maximizing the *log marginal likehood*

$$\theta_* = \underset{\theta} {\arg\max} \log p(\mathbf{y} \mid X, \theta) = \underset{\theta} {\arg\max} \left\{ -\frac{1}{2}\mathbf{y}^T \left[ K+ \sigma_n^2I \right]^{-1}\mathbf{y} -\frac{1}{2}\log det\left[K+ \sigma_n^2I\right] -\frac{T}{2}\log 2\pi \right\},$$

with $K = K(X, X)$.

#### Results
The major problem in this context was that the predictive mean $\mathbf{f}_*$ always falls back to the mean $m(\mathbf{x})$ very quickly. One solution has been to combine two squared-exponential kernels and initializing one of them to a large length-scale in order to capture the global trend. This yields to some reasonable result. However, when applying the same model to another ones ($\theta$ learned over one project and used to predict another) gives very poor performance.


### Multi-Project Regression
#### Model
Hence, our next idea is to consider $P$ projects at the same time and try to learn the hyperparameters $\theta$ over various time series. For a given project $p$, we have the following dataset $\mathcal{D}^{(p)} = \left\{ (x_i^{(p)}, y_i^{(p)}) \mid i = 1, ..., T \right\}$. Note that we have $x_i^{(p)}= x_i = i$. We combine the projects together to obtain a new dataset $\mathcal{D} = \left\{ \mathcal{D}^{(p)} \mid p = 1, ..., P \right\}$ (*multi-task learning*). We then have $X = [1, ..., T]^T$ an $(T \times 1)$ matrix of time indices and $Y = \left[\mathbf{y}^{(p)} \right]_{p=1}^P$ an $(T \times P)$ matrix of observed values per project. For a given project $p$, we are trying to predict the pledged money $\mathbf{f}_*  \equiv \mathbf{f}_*^{(p)} = \mathbf{f}_{t:T}^{(p)} = f(X_{t:T}^{(p)})$ at future time indices $X_*  \equiv X_*^{(p)} = X_{t:T}^{(p)} = X_{t:T} = [t,...,T]^T$ after observing the values $\mathbf{y}  \equiv \mathbf{y}^{(p)} = \mathbf{y}_{1:t}^{(p)}$ at time indices $X  \equiv X_{1:t} = [1, ..., t]^T$. To do so in the GP framework, we have

$$\mathbf{f}_* \mid X, \mathbf{y}, X_* \equiv \mathbf{f}_{t:T}^{(p)} \mid X_{1:t}, \mathbf{y}_{1:t}^{(p)}, X_{t:T} \sim \mathcal{N} \left( \overline{\mathbf{f}}_{t:T}^{(p)}, \text{ cov}(\mathbf{f}_{t:T}^{(p)}) \right), $$

with $\overline{\mathbf{f}}_{t:T}^{(p)} \equiv \overline{\mathbf{f}}_*$ as before and $\text{ cov}(\mathbf{f}_{t:T}^{(p)}) \equiv \text{ cov}(\mathbf{f}_*)$. The hyperparameters $\mathbf{\theta}$ of the GP are learned by maximizing the log marginal likelihood over all the projects, that is

$$\theta_* = \underset{\theta} {\arg\max} \sum_{p=1}^P \log p(\mathbf{y}^{(p)} \mid X, \theta).$$

#### Results
Again, we couldn't obtain good results with this approach, as the predictions always fall back very quickly to the mean of the GP. **[MORE DETAILS]**

### Project Classification
#### Model
We then decide to try a simpler task. Instead of trying to predict a number of (future) points after some observations, we try now to classify whether a project $p$ will be successful ($c^{(p)} = 1$) or not ($c^{(p)} = 0$). Indeed, by separating the dataset in two classes (*successul* and *failed*), we notice that they have a very different profiles (look at the mean of both classes in [sidekick-classification]) and therefore should be easy to discriminate. To do so, we train one GP using the successful projects, one using the failed projects and try to determine whether a new, partially observed project will be successful or not. We learn $\theta_s$ the hyperparameters of a GP over $P_s$ *successful* projects only and $\theta_f$ the hyperparameters of a GP over the $P_f$ *failed* projects. That is, we maximize the log marginal likelihoods

$$\theta_s = \underset{\theta} {\arg\max} \sum_{p=1}^{P_s} \log p(\mathbf{y}_s^{(p)} \mid X, \theta)$$
$$\theta_f = \underset{\theta} {\arg\max} \sum_{p=1}^{P_f} \log p(\mathbf{y}_f^{(p)} \mid X, \theta),$$

where $\mathbf{y}_s$ and $\mathbf{y}_f$ denote the observations of the successful and failed projects respectively. We then determine the success state $c^{(p)}$ of a new project $p$ with partial observations $\mathbf{y}_{1:t}^{(p)}$ as

$$c_*^{(p)} = 
\begin{cases}
    1, & \text{ if } \log p(\mathbf{y}_{1:t}^{(p)} \mid X_{1:t}, \theta_s) > \log p(\mathbf{y}_{1:t}^{(p)} \mid X_{1:t}, \theta_f) \\
    0, & \text{ otherwise }
\end{cases}.$$

#### Results

### Outputs as Inputs
#### Model
We change completely the approach. Instead of using the time as input and trying to predict the output at new time indices, we now consider the pledged money at each time step as input ($\mathbf{y}$ becomes $\mathbf{x}$) and the last time index as the output. That is, we have now a dataset $\mathcal{D} = \left\{ (\mathbf{x}^{(p)}, y^{(p)}) \mid p = 1, ..., P \right\}$ with $\mathbf{x}^{(p)} = \mathbf{y}_{1:t}^{(p)}$ and $\mathbf{y}^{(p)} = y_T^{(p)}$. We then have $X = \left[\mathbf{x}^{(p)}\right]_{p=1}^P$ a $(P \times t)$ matrix and $\mathbf{y} = \left[y_T^{(p)}\right]_{p=1}^P$ a $(P \times 1)$ target vector. The difference with the second approach is that now the features for each project are the amount of pledged money at different time step and not the same (shared) input values ($1,...,T$).

Our goal now is to predict, for a new project $p$, the final amount of pledged money $f_*^{(p)} = y_T^{(p)} = f(\mathbf{x}^{(p)}) = f(\mathbf{y}_{1:t}^{(p)})$ for the money received up to time $t$ $X_*^{(p)} = \mathbf{y}_{1:t}^{(p)}$ after observing the total pledged money for all projects $\mathbf{y} = \left[y_T^{(p)}\right]_{p=1}^P$ and the money they received up to time $t$, $X = \left[ \mathbf{y}_{1:t}^{(p)} \right]_{p=1}^P$. In the GP framework, we can compute this prediction using

$$f_* \mid X, \mathbf{y}, X_* \sim \mathcal{N}\left(\overline{f}_*, \text{ cov}(f_*)  \right) \\
\overline{f}_* = K(X_*, X) \left[ K(X, X) + \sigma_n^2I \right]^{-1}\mathbf{y} \\
\text{cov}(f_*) = K(X_*, X_*) - K(X_*, X)\left[ K(X, X) + \sigma_n^2I \right]^{-1}K(X, X_*).
$$ 

#### Results

# Some Kernels (as defined in GPy)
We assume that each data point $\mathbf{x}$ have $D$ dimensions.

## Linear

#### ARD 
$k(\mathbf{x}, \mathbf{x}') = \sum_{i=1}^{\text{D}} \sigma^2_i x_ix_i'$

#### Non-ARD 
$k(\mathbf{x}, \mathbf{x}') = \sigma^2_f \sum_{i=1}^{\text{D}} x_ix_i' = \sigma^2_f \mathbf{x}^T \mathbf{x}'$ 

## Polynomial of degree $k$

$k(\mathbf{x}, \mathbf{x}') = \sigma^2_f(\sum_{i=1}^{\text{D}}x_ix_i' + 1)^k = \sigma^2_f(\mathbf{x}^T \mathbf{x}' + 1)^k$

## Squared Exponential
$k(\mathbf{x}, \mathbf{x}') = \sigma^2_f \text{exp}\left(\frac{\left\Vert \mathbf{x}-\mathbf{x}' \right\Vert^2}{2l^2}\right)$

# GPR Algorithm

In [4]:
%matplotlib inline
from numpy.linalg import inv, cholesky
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


def k(xp, xq , l, sigma_f):
    """Covariance functions with squared exponential of length-scale l and signal noise sigma_f."""
    #return sigma_f * np.exp(-0.5 * (xp - xq)**2 / float(l**2))
    return sigma_f * xp * xq


def K(x1, x2, l=1.0, sigma_f=1.0):
    """Compute the covariance matrix from the observations x."""
    cov_matrix = np.zeros((len(x1), len(x2)))
    for i, p in enumerate(x1):
        for j, q in enumerate(x2):
            cov_matrix[i, j] = k(p, q, l, sigma_f)
    return cov_matrix


def gaussian_process_regression(x, y, x_test, k, l=1.0, sigma_n=0.0, sigma_f=1.0):
    """
    Computes a regression using Gaussian Process using observations x and y = f(x) and a covariance function k.
    
    :param x        Indices of observations
    :param y        Values at indices x (= f(x))
    :param x_test   Indices to get predicted values
    :param k        Covariance function
    :param sigma_n  Observations noise
    :return:        Mean m, variance var and log marginal likelihood lml
    """
    n = len(x)
    n_test = len(x_test)
    L = cholesky(K(x, x, l, sigma_f) + sigma_n * np.eye(n))
    L_inv = inv(L)
    a = np.dot(inv(np.transpose(L)), np.dot(L_inv, y))
    m = np.dot(K(x_test, x, l, sigma_f), a)  # Predictive mean
    v = np.dot(L_inv, K(x, x_test, l, sigma_f))
    var = K(x_test, x_test, l, sigma_f) - np.dot(np.transpose(v), v)  # Predictive variance
    lml = -0.5 * np.dot(np.transpose(y), a) - np.sum(np.diag(L)) - n * 0.3990899342  # Log maginal likelihood (last term is log2π / 2)
    return m, var + sigma_n * np.eye(n_test), lml